In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%%capture
!pip install -q "numpy<2.0" --force-reinstall
!pip install -q ultralytics opencv-python-headless tqdm matplotlib

# **1. Fix `data.yaml` (convert relative → absolute paths)**

In [3]:
import yaml
from pathlib import Path

dataset_root = Path("/kaggle/input/palletdatasetyolo8")
orig_yaml    = dataset_root / "data.yaml"

# Load original yaml
with open(orig_yaml) as f:
    data = yaml.safe_load(f)

# Fix paths (relative → absolute)
data["train"] = str(dataset_root / "train" / "images")
data["val"]   = str(dataset_root / "valid" / "images")
data["test"]  = str(dataset_root / "test"  / "images")

# Save corrected yaml
fixed_yaml = "/kaggle/working/data_fixed.yaml"
with open(fixed_yaml, "w") as f:
    yaml.dump(data, f, default_flow_style=False)

print(f"Fixed data.yaml saved: {fixed_yaml}")
print("\n--- Fixed data.yaml content ---")
!cat {fixed_yaml}

Fixed data.yaml saved: /kaggle/working/data_fixed.yaml

--- Fixed data.yaml content ---
names:
- -ECS- pallets
nc: 1
roboflow:
  license: CC BY 4.0
  project: pallet-detect-kr02r
  url: https://universe.roboflow.com/jesse-w9nkf/pallet-detect-kr02r/dataset/1
  version: 1
  workspace: jesse-w9nkf
test: /kaggle/input/palletdatasetyolo8/test/images
train: /kaggle/input/palletdatasetyolo8/train/images
val: /kaggle/input/palletdatasetyolo8/valid/images


# **2. Verify Dataset Counts**

In [4]:
for split in ["train", "valid", "test"]:
    img_cnt = len(list((dataset_root / split / "images").glob("*.*")))
    lbl_cnt = len(list((dataset_root / split / "labels").glob("*.txt")))
    print(f"{split:5} → images: {img_cnt:5}, labels: {lbl_cnt:5}")

train → images:  1755, labels:  1755
valid → images:    47, labels:    47
test  → images:    22, labels:    22


# **3. Train YOLOv8n (uses **fixed** `data.yaml`)**

In [5]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")   

results = model.train(
    data=fixed_yaml,         
    epochs=50,
    imgsz=640,
    batch=16,
    device=0,
    name="pallet_yolov8",
    project="/kaggle/working/runs",
    exist_ok=True,
    patience=10,
    save=True,
    cache=False,             
    workers=4,
    pretrained=True,
    amp=True,
    verbose=True,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    flipud=0.5, fliplr=0.5
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.228 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data_fixed.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=Fa

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         47       2900      0.929      0.926      0.973      0.719
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /kaggle/working/runs/pallet_yolov8


# **4. Find the best checkpoint**

In [6]:
import os
weight_dir = "/kaggle/working/runs/pallet_yolov8/weights"
best_pt = os.path.join(weight_dir, "best.pt")
last_pt = os.path.join(weight_dir, "last.pt")

best_model_path = best_pt if os.path.exists(best_pt) else last_pt
print(f"Best model: {best_model_path}")

# Export ONNX
model = YOLO(best_model_path)
onnx_path = best_model_path.replace(".pt", ".onnx")
try:
    model.export(format="onnx")
    print(f"ONNX exported: {onnx_path}")
except Exception as e:
    print(f"ONNX export failed: {e}")
    onnx_path = None

Best model: /kaggle/working/runs/pallet_yolov8/weights/best.pt
Ultralytics 8.3.228 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/kaggle/working/runs/pallet_yolov8/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Using Python 3.11.13 environment at: /usr
Resolved 14 packages in 258ms
Prepared 5 packages in 3.04s
Uninstalled 1 package in 196ms
Installed 5 packages in 48ms
 + coloredlogs==15.0.1
 + humanfriendly==10.0
 + onnxruntime-gpu==1.23.2
 + onnxslim==0.1.74
 - sympy==1.13.1
 + sympy==1.14.0

requirements: AutoUpdate success ✅ 3.9s
WARNING ⚠️ requirements: Re

# **5. Config**

In [ ]:
import cv2
import numpy as np
from tqdm import tqdm
from IPython.display import FileLink, display
from ultralytics import YOLO
from pathlib import Path

class Config:
    max_frames = 30
    fps        = 4
    conf_thr   = 0.20
    iou_thr    = 0.45
    # ---- NEW: smaller font settings ----
    label_font      = cv2.FONT_HERSHEY_SIMPLEX
    label_scale     = 0.45          # was 0.6 → ~25 % smaller
    label_thickness = 1             # was 2
    nopallet_scale  = 0.9           # was 1.2 → smaller “No pallet”
    nopallet_thick  = 2
cfg = Config()

# **6. Fixed paths (from the notebook above)**

In [ ]:
dataset_root    = Path("/kaggle/input/palletdatasetyolo8")
best_model_path = "/kaggle/working/runs/pallet_yolov8/weights/best.pt"   # ← adjust if needed
fixed_yaml      = "/kaggle/working/data_fixed.yaml"
onnx_path       = best_model_path.replace(".pt", ".onnx") if os.path.exists(best_model_path.replace(".pt", ".onnx")) else None

# **7. Generate 30‑frame Demo Video**

In [11]:
def create_detection_video():
    print("\nGenerating detection video …")
    model = YOLO(best_model_path)

    # ---- validation images -------------------------------------------------
    val_img_paths = list((dataset_root / "test" / "images").glob("*.*"))
    if not val_img_paths:
        print("No images found in valid/images!")
        return None

    val_images = sorted(val_img_paths)[:cfg.max_frames]
    print(f"Using {len(val_images)} validation images")

    # ---- safe draw function (smaller text) --------------------------------
    def draw_detections(img_path):
        try:
            img = cv2.imread(str(img_path))
            if img is None:
                img = np.random.randint(100, 200, (640, 640, 3), dtype=np.uint8)
            else:
                img = cv2.resize(img, (640, 640))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            res = model.predict(
                source=str(img_path),
                conf=cfg.conf_thr,
                iou=cfg.iou_thr,
                imgsz=640,
                verbose=False
            )[0]

            if res.boxes is not None and len(res.boxes) > 0:
                for box in res.boxes.data.cpu().numpy():
                    x1, y1, x2, y2, conf, cls = box
                    cls = int(cls)
                    if cls >= len(model.names):
                        cls = 0
                    if conf < cfg.conf_thr:
                        continue

                    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                    if x1 >= x2 or y1 >= y2:
                        continue

                    label = f"{model.names[cls]} {conf:.2f}"
                    color = (0, 255, 0)                     # green

                    # ---- bounding box ------------------------------------------------
                    cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color, 2)

                    # ---- label background + text (smaller) ---------------------------
                    (tw, th), _ = cv2.getTextSize(label, cfg.label_font,
                                                  cfg.label_scale, cfg.label_thickness)
                    cv2.rectangle(img_rgb,
                                  (x1, y1 - th - 8),               # a bit higher
                                  (x1 + tw, y1), color, -1)
                    cv2.putText(img_rgb, label,
                                (x1, y1 - 5),
                                cfg.label_font, cfg.label_scale,
                                (255, 255, 255), cfg.label_thickness)

            else:
                # ---- “No pallet” – also smaller ---------------------------------
                cv2.putText(img_rgb, "No pallet", (50, 80),
                            cfg.label_font, cfg.nopallet_scale,
                            (0, 255, 0), cfg.nopallet_thick)

            return img_rgb

        except Exception as e:
            print(f"Error on {img_path}: {e}")
            err = np.zeros((640, 640, 3), dtype=np.uint8)
            cv2.putText(err, "ERROR", (200, 320),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
            return err

    # ---- create frames -------------------------------------------------------
    frames = [draw_detections(p) for p in tqdm(val_images, desc="Frames")]

    # ---- write video ---------------------------------------------------------
    out_path = "/kaggle/working/pallet_detection.mp4"
    h, w = frames[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(out_path, fourcc, cfg.fps, (w, h))

    print("Writing video …")
    for fr in tqdm(frames, desc="Video"):
        writer.write(cv2.cvtColor(fr, cv2.COLOR_RGB2BGR))
    writer.release()

    size_mb = os.path.getsize(out_path) / 1e6
    print(f"Video created: {out_path} ({size_mb:.1f} MB)")

    # ---- auto‑download -------------------------------------------------------
    print("\nDownload your files:")
    display(FileLink(best_model_path))
    if onnx_path and os.path.exists(onnx_path):
        display(FileLink(onnx_path))
    display(FileLink(fixed_yaml))
    display(FileLink(out_path))

    # Colab auto‑download (if you run there)
    try:
        from google.colab import files
        files.download(out_path)
    except:
        pass

    return out_path

# ----------------------------------------------------------------------
# Run
# ----------------------------------------------------------------------
video_path = create_detection_video()


Generating detection video …
Using 22 validation images


Frames: 100%|██████████| 22/22 [00:00<00:00, 43.57it/s]


Writing video …


Video: 100%|██████████| 22/22 [00:00<00:00, 185.99it/s]

Video created: /kaggle/working/pallet_detection.mp4 (2.4 MB)

Download your files:


...

/kaggle/working/runs/pallet_yolov8/weights/best.onnx

/kaggle/working/data_fixed.yaml

/kaggle/working/pallet_detection.mp4

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>